In [1]:
from SimPEG import Mesh, Utils, EM, Maps
from pymatsolver import Pardiso
from scipy.constants import mu_0
import numpy as np
import matplotlib.pyplot as plt
from TDEM import ProblemSkyTEM
%matplotlib inline
from pyMKL import mkl_set_num_threads
from multiprocessing import Pool

In [2]:
from math import sqrt
from joblib import Parallel, delayed

In [3]:
mkl_set_num_threads(1)

In [4]:
from SimPEG import Mesh

In [1]:
!pwd

/Users/sgkang/Projects/kang-2019-3D-aem/notebooks


In [5]:
sigma_background = 1./20.
sigma_target = 1./10.
resistivity_background = 1./sigma_background
resistivity_target = 1./sigma_target
resistivity = np.array([resistivity_background, resistivity_target, resistivity_background], dtype=float)
thickness = np.array([50, 40], dtype=float)
source_area = 536.36
# data_dir = "/Users/sgkang/Dropbox/Stanford/Resolution/aarhusinv/em1dinv_examples/notebooks/"
data_dir = "/Users/seogi/Dropbox/Stanford/Resolution/aarhusinv/em1dinv_examples/notebooks/"
waveform_hm = np.loadtxt(data_dir+"HM_butte_304.txt")
time_gates_hm = np.loadtxt(data_dir+"HM_butte_304_gates")[7:,:] * 1e-6
waveform_lm = np.loadtxt(data_dir+"LM_butte_304.txt")
time_gates_lm = np.loadtxt(data_dir+"LM_butte_304_gates")[8:,:] * 1e-6

time_input_currents_HM = waveform_hm[:,0]
input_currents_HM = waveform_hm[:,1]
time_input_currents_LM = waveform_lm[:,0]
input_currents_LM = waveform_lm[:,1]
    
time_LM = time_gates_lm[:,3] - waveform_lm[:,0].max()
time_HM = time_gates_hm[:,3] - waveform_hm[:,0].max()
source_area = 536.36

In [6]:
from discretize import utils
rxloc = np.array([13.25, 0., 30.+2.])
srcloc = np.array([0., 0., 30.])
xyz = np.vstack((rxloc, srcloc))
x = np.linspace(-1200., 1200.)
y = np.linspace(-50., 50.)
dem = Utils.ndgrid(x,y,np.r_[0.])
h = [10., 10., 5.]
mesh_global = utils.mesh_builder_xyz(
    dem,
    h,
    padding_distance=[[2000., 2000.], [1000, 1000], [2000., 0.]],
    base_mesh=None,
    depth_core=None,
    expansion_factor=1.3,
    mesh_type='tree'
)

mesh_global = utils.refine_tree_xyz(
    mesh_global,
    dem,
    method='surface',
    octree_levels=[4, 5, 5],
    octree_levels_padding=None,
    finalize=False,
    min_level=0,
    max_distance=np.inf,    
)

x = np.linspace(-2000., 2000.)
y = np.linspace(-50., 50.)
dem = Utils.ndgrid(x,y,np.r_[0.])


mesh_global = utils.refine_tree_xyz(
    mesh_global,
    dem,
    method='surface',
    octree_levels=[0, 1, 1],
    octree_levels_padding=None,
    finalize=True,
    min_level=0,
    max_distance=np.inf,    
)

In [7]:
x = np.arange(1001)*1 - 1000.
y = np.r_[0.]
z = np.r_[30.]
srclocs = Utils.ndgrid(x, y, z)
rxlocs = Utils.ndgrid(x-13.25, y, z+2.)
n_src = srclocs.shape[0]
actv_global = mesh_global.gridCC[:,2]<0.


In [8]:
# mesh_global.plotSlice(np.log10(1./sigma), normal='X', grid=True, clim=(np.log10(10), np.log10(50)))
# plt.plot(srclocs[:,0], srclocs[:,2], 'r.')
# plt.xlim(-2000, 2000)
# plt.ylim(-300, 100)
# # plt.gca().set_aspect(5)

In [9]:
# mesh_global.plotSlice(np.log10(1./sigma), normal='z', grid=True, clim=(np.log10(10), np.log10(50)), ind=255)
# plt.plot(srclocs[:,0], srclocs[:,1], 'r.')
# plt.xlim(-200, 200)
# plt.ylim(-200, 200)
# # plt.gca().set_aspect(5)

In [10]:
rxloc = np.array([13.25, 0., 30.+2.])
srcloc = np.array([0., 0., 30.])

def create_local_mapping(srcloc, rxloc, mesh_global, actv_global):
    xyz = np.vstack((rxloc, srcloc))
    print (srcloc[0], srcloc[1])
    x = np.linspace(-100., 100.) + srcloc[0]
    y = np.linspace(-20., 20.) + srcloc[1]
    dem = Utils.ndgrid(x,y,np.r_[0.])
    h = [10., 10., 5.]
    mesh_local = utils.mesh_builder_xyz(
        dem,
        h,
        padding_distance=[[2000., 2000.], [2000., 2000.], [2000., 2000.]],
        base_mesh=None,
        depth_core=None,
        expansion_factor=1.3,
        mesh_type='tree'
    )

    mesh_local = utils.refine_tree_xyz(
        mesh_local,
        dem,
        method='surface',
        octree_levels=[1, 0, 5],
        octree_levels_padding=None,
        finalize=False,
        min_level=0,
        max_distance=np.inf,    
    )


    mesh_local = utils.refine_tree_xyz(
        mesh_local,
        xyz,
        method='radial',
        octree_levels=[2, 0, 0],
        octree_levels_padding=None,
        finalize=True,
        min_level=1,
        max_distance=np.inf,    
    )
    # Assume flat topo
    actv_local = mesh_local.gridCC[:,2]<0.
    tile_map = Maps.Tile((mesh_global, actv_global), (mesh_local, actv_local))    
    act_map = Maps.InjectActiveCells(mesh_local, tile_map.activeLocal, valInactive=1e-8)
    mapping = act_map * tile_map 
    return mapping

In [11]:
def run_simulation(args):
    mapping, srcloc, rxloc, sigma, time_HM, time_LM, time_input_currents_HM, input_currents_HM, time_input_currents_LM, input_currents_LM = args
    map_t = mapping.maps[1]
    mesh_local = map_t.meshLocal  
    actv_local = mesh_local.gridCC[:,2]<0.
    rx = EM.TDEM.Rx.Point_dbdt(rxloc, np.logspace(np.log10(1e-6), np.log10(1e-2), 31), 'z')
    src = EM.TDEM.Src.MagDipole([rx], waveform=EM.TDEM.Src.StepOffWaveform(), loc=srcloc)
    survey = EM.TDEM.Survey([src])
    prb = ProblemSkyTEM(mesh_local, sigmaMap=mapping, verbose=False)
    dts = np.diff(np.logspace(-6, -1, 50))
    prb.timeSteps = [(3e-7, 6),(1e-6, 5),(2e-6, 5),(5e-6, 5),(1e-5, 5),(2e-5, 5),(5e-5, 5),(1e-4, 5),(2e-4, 5),(5e-4, 5),(1e-3, 15)]
    prb.Solver = Pardiso
    prb.pair(survey)      
    data = prb.simulate(
            sigma[mapping.maps[1].actvGlobal],
            time_HM,
            time_LM,
            time_input_currents_HM, 
            input_currents_HM,
            time_input_currents_LM, 
            input_currents_LM,    
    )  
    return data

def input_args(ii):    
    args = (
        mappings[ii],
        srclocs[ii,:], 
        rxlocs[ii,:],
        sigmas[0],
        time_HM,
        time_LM,
        time_input_currents_HM,
        input_currents_HM,
        time_input_currents_LM,
        input_currents_LM,
    )    
    return args

In [12]:
sigmas = []
ls = [20, 40, 80, 160, 240, 320]
for l in ls:
    sigma = np.ones(mesh_global.nC) * 1e-8
    sigma[actv_global] = sigma_background
    z = 0.
    h = 20
#     l = 400
    inds = np.logical_and(mesh_global.gridCC[:,2]<-(z), mesh_global.gridCC[:,2]>-(z+h))
    sigma[inds] = sigma_target
    inds_gap = np.logical_and(mesh_global.gridCC[:,0]<l/2, mesh_global.gridCC[:,0]>-l/2) & inds
    sigma[inds_gap] = sigma_background
    sigma[~actv_global] = 1e-8
    sigmas.append(sigma)

In [13]:
mappings = []
for ii in range(n_src):
    mappings.append(create_local_mapping(srclocs[ii,:], rxlocs[ii,:], mesh_global, actv_global))    

-1000.0 0.0
-999.0 0.0
-998.0 0.0
-997.0 0.0
-996.0 0.0
-995.0 0.0
-994.0 0.0
-993.0 0.0
-992.0 0.0
-991.0 0.0
-990.0 0.0
-989.0 0.0
-988.0 0.0
-987.0 0.0
-986.0 0.0
-985.0 0.0
-984.0 0.0
-983.0 0.0
-982.0 0.0
-981.0 0.0
-980.0 0.0
-979.0 0.0
-978.0 0.0
-977.0 0.0
-976.0 0.0
-975.0 0.0
-974.0 0.0
-973.0 0.0
-972.0 0.0
-971.0 0.0
-970.0 0.0
-969.0 0.0
-968.0 0.0
-967.0 0.0
-966.0 0.0
-965.0 0.0
-964.0 0.0
-963.0 0.0
-962.0 0.0
-961.0 0.0
-960.0 0.0
-959.0 0.0
-958.0 0.0
-957.0 0.0
-956.0 0.0
-955.0 0.0
-954.0 0.0
-953.0 0.0
-952.0 0.0
-951.0 0.0
-950.0 0.0
-949.0 0.0
-948.0 0.0
-947.0 0.0
-946.0 0.0
-945.0 0.0
-944.0 0.0
-943.0 0.0
-942.0 0.0
-941.0 0.0
-940.0 0.0
-939.0 0.0
-938.0 0.0
-937.0 0.0
-936.0 0.0
-935.0 0.0
-934.0 0.0
-933.0 0.0
-932.0 0.0
-931.0 0.0
-930.0 0.0
-929.0 0.0
-928.0 0.0
-927.0 0.0
-926.0 0.0
-925.0 0.0
-924.0 0.0
-923.0 0.0
-922.0 0.0
-921.0 0.0
-920.0 0.0
-919.0 0.0
-918.0 0.0
-917.0 0.0
-916.0 0.0
-915.0 0.0
-914.0 0.0
-913.0 0.0
-912.0 0.0
-911.0 0.0
-910.0 0.

-255.0 0.0
-254.0 0.0
-253.0 0.0
-252.0 0.0
-251.0 0.0
-250.0 0.0
-249.0 0.0
-248.0 0.0
-247.0 0.0
-246.0 0.0
-245.0 0.0
-244.0 0.0
-243.0 0.0
-242.0 0.0
-241.0 0.0
-240.0 0.0
-239.0 0.0
-238.0 0.0
-237.0 0.0
-236.0 0.0
-235.0 0.0
-234.0 0.0
-233.0 0.0
-232.0 0.0
-231.0 0.0
-230.0 0.0
-229.0 0.0
-228.0 0.0
-227.0 0.0
-226.0 0.0
-225.0 0.0
-224.0 0.0
-223.0 0.0
-222.0 0.0
-221.0 0.0
-220.0 0.0
-219.0 0.0
-218.0 0.0
-217.0 0.0
-216.0 0.0
-215.0 0.0
-214.0 0.0
-213.0 0.0
-212.0 0.0
-211.0 0.0
-210.0 0.0
-209.0 0.0
-208.0 0.0
-207.0 0.0
-206.0 0.0
-205.0 0.0
-204.0 0.0
-203.0 0.0
-202.0 0.0
-201.0 0.0
-200.0 0.0
-199.0 0.0
-198.0 0.0
-197.0 0.0
-196.0 0.0
-195.0 0.0
-194.0 0.0
-193.0 0.0
-192.0 0.0
-191.0 0.0
-190.0 0.0
-189.0 0.0
-188.0 0.0
-187.0 0.0
-186.0 0.0
-185.0 0.0
-184.0 0.0
-183.0 0.0
-182.0 0.0
-181.0 0.0
-180.0 0.0
-179.0 0.0
-178.0 0.0
-177.0 0.0
-176.0 0.0
-175.0 0.0
-174.0 0.0
-173.0 0.0
-172.0 0.0
-171.0 0.0
-170.0 0.0
-169.0 0.0
-168.0 0.0
-167.0 0.0
-166.0 0.0
-165.0 0.0

In [14]:
# %%time
# results = []
# for ii in range(x.size):
#     args = input_args(ii)
#     results.append(run_simulation(args))

In [15]:
%%time
results = Parallel(n_jobs=8)(delayed(run_simulation)(input_args(ii)) for ii in range(x.size)) 

C:\Users\seogi\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Wall time: 2min 52s
